# Digit Recognizer
## Classify handwritten digits using the famous MNIST data

### train.csv
https://www.kaggle.com/c/digit-recognizer/download/train.csv
### test.csv
https://www.kaggle.com/c/digit-recognizer/download/test.csv

The data files train.csv and test.csv contain gray-scale images of hand-drawn digits, from zero through nine.

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixelx is located on row i and column j of a 28 x 28 matrix, (indexing by zero).

In [1]:
# kelas
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.optimizers import SGD
from keras.utils import np_utils
# basic
import pandas as pd
import numpy as np
from pylab import *

Using TensorFlow backend.


In [2]:
minst = pd.read_csv("./Digit Recognizer/train.csv")

def get_df(minst, train_rate=.75):

    
    minst['is_train'] = np.random.uniform(0, 1, len(minst)) <= train_rate
    train, test = minst[minst['is_train']==True], minst[minst['is_train']==False]
    
    X_label = train.columns[1:-1]
    y_label = train.columns[0]
    
    X_train, y_train = train[X_label], train[y_label]
    X_test, y_test = test[X_label], test[y_label]
    
    
    
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    
    return np.array(X_train), y_train, np.array(X_test), y_test

X_train, y_train, X_test, y_test = get_df(minst)

# (X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0],28,28,1)
X_test = X_test.reshape(X_test.shape[0],28,28,1)
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [21]:
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

def network():
    model = Sequential()
    model.add(Convolution2D(32, kernel_size[0], kernel_size[1], input_shape = (28,28,1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(16, kernel_size[0], kernel_size[1]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    
    model.add(Dropout(0.25))
    model.add(Flatten())

    model.add(Dense(128, init='uniform'))
    model.add(Activation('relu'))
    model.add(Dense(128, init='uniform'))
    model.add(Activation('relu'))
    model.add(Dense(10, init='uniform'))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer="Adadelta",
              metrics=['accuracy'])
    return model

In [22]:
# train
nn = network()
history = nn.fit(X_train, y_train, nb_epoch=20, batch_size=64, verbose=1, validation_data=(X_test, y_test))
nn.evaluate(X_test, y_test, batch_size=16)

Train on 31349 samples, validate on 10651 samples
Epoch 1/20
31349/31349 [==============================] - 7s - loss: 0.6108 - acc: 0.7937 - val_loss: 0.1429 - val_acc: 0.9565
Epoch 2/20
31349/31349 [==============================] - 7s - loss: 0.1354 - acc: 0.9566 - val_loss: 0.0952 - val_acc: 0.9722
Epoch 3/20
31349/31349 [==============================] - 7s - loss: 0.1048 - acc: 0.9663 - val_loss: 0.0798 - val_acc: 0.9757
Epoch 4/20
31349/31349 [==============================] - 7s - loss: 0.0917 - acc: 0.9710 - val_loss: 0.0723 - val_acc: 0.9784
Epoch 5/20
31349/31349 [==============================] - 7s - loss: 0.0790 - acc: 0.9744 - val_loss: 0.0672 - val_acc: 0.9786
Epoch 6/20
31349/31349 [==============================] - 7s - loss: 0.0709 - acc: 0.9774 - val_loss: 0.0598 - val_acc: 0.9825
Epoch 7/20
31349/31349 [==============================] - 7s - loss: 0.0652 - acc: 0.9793 - val_loss: 0.0565 - val_acc: 0.9827
Epoch 8/20
31349/31349 [==============================] - 7s 

[0.040939871669025032, 0.98798234907520421]

In [20]:
for layer in nn.layers:
    print(layer.output_shape)

(None, 26, 26, 32)
(None, 26, 26, 32)
(None, 13, 13, 32)
(None, 11, 11, 32)
(None, 11, 11, 32)
(None, 5, 5, 32)
(None, 5, 5, 32)
(None, 800)
(None, 128)
(None, 128)
(None, 128)
(None, 128)
(None, 10)
(None, 10)


In [5]:
test  = pd.read_csv("./Digit Recognizer/test.csv")
_test = np.array(test)
_test = _test.reshape(_test.shape[0],28,28,1)
result = nn.predict(_test)

In [6]:
l = []
for line in result:
    m = np.amax(line)
    for i in range(10):
        if line[i]==m:
            l.append(i)
            
resu = np.array(l)

submit = pd.DataFrame(columns=['ImageId', 'Label'])
submit['Label'] = resu
submit['ImageId'] = [i for i in range(1, len(resu) + 1)]

In [7]:
submit.to_csv('./Digit Recognizer/submit.csv', index=False)